# Redes Neurais

Redes neurais podem ser construídas usando o pacote `torch.nn`.

As redes neurais `nn` (neural-networks) depende do [`autograd`](./AUTOGRAD_Diferenciacao_Automatica.ipynb) para definir modelos e diferenciá-los. Um `nn.Module` contém camadas e um método `forward(input)` que retorna a saída `output`.

Por exemplo, veja esta rede neural que classifica imagens de dígito:

![mnist](./img/mnist.png)

## ConvNet - Rede Convolutiva

É uma rede simples _feed-forward_ (sem realimentação). 
Esta recebe o _input_ (entrada), alimenta através das diversas camadas, uma após a outra, e retorna um _output_ (saída).

Um típico procedimento de treinamento para uma rede neural ocorre da seguinte forma:
* Definir a rede neural com parâmetros aprendíveis (ou pesos);
* Iterar sobre um dataset de _inputs_ (entradas);
* Processar o _input_ através da rede;
* Computar a perda / _loss_ (quão distante está a saída da resposta correta);
* Retropropagar os gradientes nos parâmetros da rede;
* Atualizar os pesos da rede, ticamente usando uma regra simples de atualização: `weight = weight - learning_rate * gradient` (`peso = peso - taxa_aprendizage * gradiente`).

## Definir a rede

Vamos definir esta rede:

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 canal de entrada de imagem, 6 canais de saída,
        # convolução com kernel quadrado 3x3.
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # uma operação afim: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120) # 5*5 da dimensão da imagem.
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        # Max pooling sobre janela 2x2
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # Se a dimensão for quadrada, pode-se apenas especificar
        # um único número.
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # todas as dimensões exceto a dimensão do batch (lote).
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


Devemos apenas que definir a função `forward`, e a função `backward` (onde os gradientes são computados) é automaticamente definida usando o `autograd`. Podemos usar quaisquer operações de Tensor na função `forward`.

Os parâmetros aprendíveis de um modelo são retornados pelo `net.parameters()`.

In [21]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # pesos da conv1

10
torch.Size([6, 1, 3, 3])


Vamos tentar uma entrada aleatória 32x32.
> É esperado nesta rede (LeNet) uma entrada de dimensões 32x32. Para usar esta rede no _dataset_ MNIST, favor redimensionar as imagens do mesmo para 32x32.

In [22]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0703,  0.0442, -0.0490, -0.2371,  0.0373, -0.0442, -0.0313, -0.1098,
          0.1088,  0.0491]], grad_fn=<AddmmBackward>)


Zere os _buffers_ de gradiente de todos parâmetros e retropropagações com gradientes aleatórios:

In [23]:
net.zero_grad()
out.backward(torch.randn(1, 10))

> `torch.nn` suporta apenas _mini-batches_. O pacote `torch.nn` inteiro apenas suporta entradas que são _mini-batches_ (mini-lotes) de amostras, e não amostra única.

> Por exemplo, `nn.Conv2d` receberá um Tensor 4D de `nAmostras x nCanais x Altura x Largura`.

> Se tiver uma amostra única, apenas use `input.unsqueeze(0)` para adicionar uma falsa dimensão de lote.

Antes de prosseguir, vamos recapitular todos os conteúdos vistos até agora.

## Recapituação

* `torch.Tensor` - um _array_ multi-dimensional com suporte ao às operações de autograd, como `backward()`. Também mantém o gradiente com respeito ao tensor.
* `nn.Module` - _Neural Network Module_. Meio conveniente de encapsular parâmetros, com auxílio à movimentação dos mesmos ao GPU, exportando, carregando, etc.
* `nn.Parameter` - um tipo de Tensor que é automaticamente registrado como um parâmetro quando atribuido como um atributo de um `Module`.
* `autograd.Function` - Implementa as definições de _forward_ e _backward_ de uma operação autograd. Toda operação Tensor cria pelo menos um nó de `Function`que conecta à funções que criam um Tensor e codifica seu histórico.

## _Loss Function_ (Função de Perda)

Uma função de perda pega o par (_output_, _target_) como entradas e computa um valor que estima quão distante o _output_ (saída) está do _target_ (alvo).

Há diversas funções de perda diferentes no pacote `nn`. Uma função de perda simples é: `nn.MSELoss`, que computa o erro quadrático médio entre o _output_ e o _target_.

Por exemplo:

In [24]:
output = net(input)
target = torch.randn(10)  # um alvo aleatório, por exemplo.
target = target.view(1, -1)  # redimensiona para a mesma forma da saída.
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.9116, grad_fn=<MseLossBackward>)


Agora, se seguimos `loss` na direção _backward_ (reversa), usando seu atributo `.grad_fn`, veremos um grafo de computações parecido com:
> input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d <br/>
-> view -> linear -> relu -> linear -> relu -> linear <br/>
-> MSELoss <br/>
-> loss

Então, quando chamamos `loss.backward()`, o grafo inteiro é diferenciado com relação à perda, e todos Tensores no grafo que tem `requires_grad=True` terão seus Tensores `.grad` acumulados com o gradiente.

Para ilustrar, vamos seguir alguns passos _backwards_:

In [25]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLu

## Retropropagação (_Backprop_)

Para retropropagar o erro, tudo que devemos fazer é `loss.backward()`. Devemos limpar os gradientes existentes ou, caso contrário, os gradientes serão acumulados aos gradientes existentes.

Agora chamamos `loss.backward()`. Observe os gradientes do _bias_ (viés_ na `conv1` antes e depois do `backward`.

In [26]:
net.zero_grad()  # zera os buffers de gradiente de todos os parâmetros

print("conv1.bias.grad antes da retropropagação.")
print(net.conv1.bias.grad)

loss.backward()

print("conv1.bias.grad após a retropropagação.")
print(net.conv1.bias.grad)

conv1.bias.grad antes da retropropagação.
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad após a retropropagação.
tensor([-0.0117, -0.0117,  0.0243, -0.0102,  0.0217,  0.0124])


Agora vimos como usar as funções de perda.

## Atualização dos pesos

A regra mais simples de atualização usada na prática é o _Stochastic Gradient Descent_ (Gradiente Descendente Estocástico - SGD):

$$peso = peso - taxa\_aprendizagem * gradiente$$

Podemos implementá-lo usando código simples em Python:

In [27]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

No entanto, como usamos redes neurais, queremos usar várias regras diferentes de atualização, como _SGD_, _Nesterov-SGD_, _Adam_, _RMSProp_, etc. Para isso, há um pequeno pacote: `torch.optim` que implementa todos esses métodos.

In [28]:
import torch.optim as optim

# cria seu otimizador
optimizer = optim.SGD(net.parameters(), lr=0.01)

# no seu loop de treinamento
optimizer.zero_grad()  # zera os buffers de gradiente
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()  # faz a atualização.

> Observe como os _buffers_ de gradiente tiveram que ser zerados manualmente usando `optimizer.zero_grad()`. Isso porque os gradientes são acumulados, conforme explicado anteriormente na sessão [Retropropagação](##retropropagação-(_Backprop_)).

> Referência: [NEURAL NETWORK](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py)